In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
# Importar e limpar os dados
df = pd.read_csv("fcc-forum-pageviews.csv", parse_dates=['date'], index_col='date')

In [3]:
df_limpo = df[
    (df['value'] >= df['value'].quantile(0.025)) & 
    (df['value'] <= df['value'].quantile(0.975))
]

In [4]:
def draw_line_plot():
    # Copiar o dataframe limpo
    df_plot = df_limpo.copy()

    # Criar figura e gráfico
    fig, ax = plt.subplots(figsize=(15, 5))
    ax.plot(df_plot.index, df_plot['value'], color='red', linewidth=1)
    ax.set_title('Daily freeCodeCamp Forum Page Views 5/2016-12/2019')
    ax.set_xlabel('Date')
    ax.set_ylabel('Page Views')

    # Salvar figura
    fig.savefig('line_plot.png')
    return fig

In [5]:
def draw_bar_plot():
    # Preparar os dados
    df_bar = df_limpo.copy()
    df_bar['year'] = df_bar.index.year
    df_bar['month'] = df_bar.index.month
    df_bar['month_name'] = df_bar.index.strftime('%B')

    # Agrupar por ano e mês e calcular média
    df_grouped = df_bar.groupby(['year', 'month_name'])['value'].mean().unstack()

    # Garantir ordem correta dos meses
    ordered_months = ['January', 'February', 'March', 'April', 'May', 'June',
                      'July', 'August', 'September', 'October', 'November', 'December']
    df_grouped = df_grouped[ordered_months]

    # Criar gráfico de barras
    fig = df_grouped.plot(kind='bar', figsize=(15, 7)).figure
    plt.xlabel('Years')
    plt.ylabel('Average Page Views')
    plt.legend(title='Months')
    plt.tight_layout()

    # Salvar figura
    fig.savefig('bar_plot.png')
    return fig

In [6]:
def draw_box_plot():
    # Preparar dados
    df_box = df_limpo.copy()
    df_box.reset_index(inplace=True)
    df_box['year'] = df_box['date'].dt.year
    df_box['month'] = df_box['date'].dt.strftime('%b')
    df_box['month_num'] = df_box['date'].dt.month

    # Ordenar meses corretamente
    df_box = df_box.sort_values('month_num')

    # Criar figura e subplots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

    # Boxplot por ano
    sns.boxplot(x='year', y='value', data=df_box, ax=axes[0])
    axes[0].set_title('Year-wise Box Plot (Trend)')
    axes[0].set_xlabel('Year')
    axes[0].set_ylabel('Page Views')

    # Boxplot por mês
    sns.boxplot(x='month', y='value', data=df_box, ax=axes[1])
    axes[1].set_title('Month-wise Box Plot (Seasonality)')
    axes[1].set_xlabel('Month')
    axes[1].set_ylabel('Page Views')

    # Salvar figura
    fig.savefig('box_plot.png')
    return fig